# unify station

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

# 按距离同一所有站点编号

In [2]:
all_files = os.listdir('station')
all_station = set([x.split('.')[0].split('-')[0] for x in all_files])
bus = pd.DataFrame()
for i in all_station:
    path = 'station/'+str(i)+'.txt';
    temp = pd.read_csv(path, header=None,sep=',')
    temp['line'] = i
    bus = pd.concat([bus, temp])
bus.columns = ['up','no','loc1','loc2','line']
bus = bus[['line','up','no','loc1','loc2']]
info = {}
for item in bus.iterrows():
    temp = item[1]
    line = int(temp['line'])
    loc1 = temp['loc1']
    loc2 = temp['loc2']
    if isinstance(loc1, str):
        loc1 = float(loc1.strip())
    if isinstance(loc2, str):
        loc2 = float(loc2.strip())
    
    if np.isnan(loc1) and np.isnan(loc2):
        continue
        
    info[(line,temp['up'],temp['no'])] = (loc1, loc2)
from math import sin, cos, sqrt, atan2, radians,acos,pi

def distance(coord1, coord2):
    R = 6373.0
    lat1 = radians(coord1[0])
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d

info_copy = list(info.copy().items())
s = []
for i in info_copy:
    temp = set()
    
    k = i[0]
    v = i[1]
    
    temp.add(k)
    info_copy.remove(i)
    
    for j in info_copy:
        j_k = j[0]
        j_v = j[1]
        if i != j:
            if distance(v, j_v) < 0.05:
                temp.add(j_k)
                info_copy.remove(j)
    
    s.append(temp)
    
res = {}
for i,v in enumerate(s):
    for k in v:
        res[k] = i

In [4]:
with open('station_map.pkl','wb') as f:
    pickle.dump(res,f)
with open('./data/train-id4-crowd-grid3.txt', 'rb') as data_file:
    line = pickle.load(data_file)
with open('./data/test-id4-crowd-grid3.txt', 'rb') as data_file:
    test = pickle.load(data_file)
line['Source_Station_encode'] = line.apply(lambda x:station_map.get((x['O_LINENO'], x['O_UP'], x['Source_Station']), -1), axis=1)
line['Target_Station_encode'] = line.apply(lambda x:station_map.get((x['O_LINENO'], x['O_UP'], x['Target_Station']), -1), axis=1)
test['Source_Station_encode'] = test.apply(lambda x:station_map.get((x['O_LINENO'], x['O_UP'], x['Source_Station']),-1), axis=1)
test['Target_Station_encode'] = test.apply(lambda x:station_map.get((x['O_LINENO'], x['O_UP'], x['Target_Station']),-1), axis=1)

"---save---"
with open('./data/train-id4-crowd-grid4.txt', 'wb') as data_file:
    pickle.dump(train, data_file)
with open('./data/test-id4-crowd-grid4.txt', 'wb') as data_file:
    pickle.dump(test, data_file)